## prepro

In [4]:
import numpy as np
import pandas as pd
import nltk
import re
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()
def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]
    
    return " " .join(text)

def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):
    
    text = text.split()

    text=[y.lower() for y in text]
    
    return " " .join(text)

def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan
            
def normalize_text(df):
    df.content=df.content.apply(lambda text : lower_case(text))
    df.content=df.content.apply(lambda text : remove_stop_words(text))
    df.content=df.content.apply(lambda text : Removing_numbers(text))
    df.content=df.content.apply(lambda text : Removing_punctuations(text))
    df.content=df.content.apply(lambda text : Removing_urls(text))
    df.content=df.content.apply(lambda text : lemmatization(text))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

df = pd.read_csv('dataset/emotions.csv')
test = df.copy()
# drop the tweet_id column
df.drop('tweet_id', axis=1, inplace=True)
index = df[df['content'].duplicated() == True].index
df.drop(index, axis = 0, inplace = True)
df.reset_index(inplace=True, drop = True)
df = normalize_text(df)

# current directory: 
dir = os.getcwd()
path = os.path.join(dir, 'dataset', 'emotions-prepro.csv')
df.to_csv(path, index=False)



# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tibed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tibed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# train test split

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset/emotions-prepro.csv')

train_test_split = 0.2

# shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

# split the data into train and test csv files
msk = np.random.rand(len(df)) < train_test_split
train = df[~msk]
test = df[msk]

# save the train and test dataframes to csv files
train.to_csv('dataset/emotions-train.csv', index=False)
test.to_csv('dataset/emotions-test.csv', index=False)



In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('dataset/emotions-prepro.csv')

# Convert 'content' column to string type
df['content'] = df['content'].astype(str)

X = df.content
y = df.sentiment

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

tokenizer = Tokenizer(oov_token='UNK')
test = pd.concat([X_train,X_test])
tokenizer.fit_on_texts(test)

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

maxlen = max([len(t) for t in df['content']])

X_train = pad_sequences(sequences_train, maxlen=maxlen, truncating='pre')
X_test = pad_sequences(sequences_test, maxlen=maxlen, truncating='pre')

vocabSize = len(tokenizer.index_word) + 1
print(f"Vocabulary size = {vocabSize}")


Vocabulary size = 43404


# train

In [9]:
def encodeLabels(y_train, y_test):
    """
    Encode the labels
    :param y_train: The training labels
    :param y_test: The testing labels
    :return: The encoded labels and the labels
    """

    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    labels = le.classes_
    print(f"Labels: {labels} -- {le.transform(labels)}")
    return y_train, y_test, labels

In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.preprocessing import LabelEncoder

df_training = pd.read_csv('dataset/emotions-train.csv')
df_test = pd.read_csv('dataset/emotions-test.csv')

# get the features and labels
X_train = df_training.content.astype(str)
X_test = df_test.content.astype(str)
y_train = df_training.sentiment
y_test = df_test.sentiment

# encode the labels
y_train, y_test, labels = encodeLabels(y_train, y_test)

# tokenize the data
tokenizer = Tokenizer(oov_token='UNK')
print("type",type(X_train))
full_text = pd.concat([X_train,X_test])
tokenizer.fit_on_texts(pd.concat([X_train,X_test]))

# convert text to sequences
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

# pad the sequences
maxlen = max([len(t) for t in pd.concat([X_train,X_test])])

X_train = pad_sequences(sequences_train, maxlen=maxlen, truncating='pre')
X_test = pad_sequences(sequences_test, maxlen=maxlen, truncating='pre')

vocabSize = len(tokenizer.index_word) + 1
print(f"Vocabulary size = {vocabSize}")

glove_file = os.path.join('dataset', 'glove.6B.200d.txt')
num_tokens = vocabSize
embedding_dim = 200
hits = 0
misses = 0

embeddings_index = {}

with open(glove_file, encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print("Found %s word vectors." % len(embeddings_index))

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Labels: ['anger' 'boredom' 'empty' 'enthusiasm' 'fun' 'happiness' 'hate' 'love'
 'neutral' 'relief' 'sadness' 'surprise' 'worry'] -- [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
type <class 'pandas.core.series.Series'>
Vocabulary size = 43404
Found 400000 word vectors.
Converted 20279 words (23124 misses)


NameError: name 'vocabSize' is not defined

In [13]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional

adam = Adam(learning_rate=0.005)

model = Sequential()
model.add(Embedding(vocabSize, 200, input_length=X_train.shape[1], weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 136, 200)          8680800   
                                                                 
 bidirectional (Bidirectiona  (None, 136, 256)         336896    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 136, 128)         164352    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 13)                1677      
                                                        

In [14]:
# use gpu if available
from tensorflow.keras.callbacks import ModelCheckpoint
modelCheckpoint = ModelCheckpoint('model/model.h5', save_best_only=True)

history = model.fit(X_train, y_train, batch_size=256, epochs=1, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001), modelCheckpoint])

25/25 [==============================] - 115s 4s/step - loss: 2.1606 - accuracy: 0.2421 - val_loss: 2.0206 - val_accuracy: 0.3044


In [16]:
predictions = model.predict(X_test, batch_size=256, verbose=1)

125/125 [==============================] - 51s 412ms/step


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
classifications = predictions.argmax(axis=1)
report = classification_report(y_test.argmax(axis=1), classifications)
print(report)
cf_matrix = confusion_matrix(y_test.argmax(axis=1), classifications)
print(cf_matrix)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.00      0.00      0.00       150
           2       0.00      0.00      0.00       663
           3       0.00      0.00      0.00       627
           4       0.00      0.00      0.00      1402
           5       0.25      0.16      0.20      4192
           6       0.00      0.00      0.00      1060
           7       0.26      0.37      0.31      2995
           8       0.29      0.65      0.40      6862
           9       0.00      0.00      0.00      1221
          10       0.00      0.00      0.00      4091
          11       0.00      0.00      0.00      1739
          12       0.33      0.45      0.38      6740

    accuracy                           0.29     31830
   macro avg       0.09      0.13      0.10     31830
weighted avg       0.19      0.29      0.22     31830

[[   0    0    0    0    0    3    0    4   46    0    0    0   35]
 [   0    0

c:\Users\tibed\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tibed\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tibed\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
np.save(os.path.join("model/"+'test-report.npy'), report)

In [21]:
# load report
report = np.load(os.path.join("model/"+'test-report.npy'), allow_pickle='TRUE').item()
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.00      0.00      0.00       150
           2       0.00      0.00      0.00       663
           3       0.00      0.00      0.00       627
           4       0.00      0.00      0.00      1402
           5       0.25      0.16      0.20      4192
           6       0.00      0.00      0.00      1060
           7       0.26      0.37      0.31      2995
           8       0.29      0.65      0.40      6862
           9       0.00      0.00      0.00      1221
          10       0.00      0.00      0.00      4091
          11       0.00      0.00      0.00      1739
          12       0.33      0.45      0.38      6740

    accuracy                           0.29     31830
   macro avg       0.09      0.13      0.10     31830
weighted avg       0.19      0.29      0.22     31830



In [27]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.savefig(os.path.join("test"+'-loss.png'))
plt.clf()

# plot the accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.savefig(os.path.join("test"+'-accuracy.png'))
plt.clf()

<Figure size 640x480 with 0 Axes>